<a href="https://colab.research.google.com/github/makaveli10/stylegan3/blob/main/face_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Morphing - StyleGAN3
 Network details
 

In [ ]:
NETWORK = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl'
STEPS = 150
FPS = 30
FREEZE_STEPS = 30

In [ ]:
# @title CodeFormer upsampling
!git clone https://github.com/sczhou/CodeFormer.git
%cd CodeFormer
!git checkout e501cd0
import sys
sys.path.insert(0, "/content/CodeFormer")
!pip install -r requirements.txt
!python basicsr/setup.py develop
%cd ..
!python CodeFormer/scripts/download_pretrained_models.py facelib
!python CodeFormer/scripts/download_pretrained_models.py CodeFormer

## Upload images

In [ ]:
import os
from google.colab import files

uploaded = files.upload()

if len(uploaded) != 1:
  print("Upload exactly 1 file for source.")
else:
  for k, v in uploaded.items():
    SOURCE, ext = os.path.splitext(k)
    SOURCE_NAME = k

In [ ]:
uploaded = files.upload()

if len(uploaded) != 1:
  print("Upload exactly 1 file for target.")
else:
  for k, v in uploaded.items():
    TARGET, ext = os.path.splitext(k)
    TARGET_NAME = k

In [ ]:
!wget http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_5_face_landmarks.dat.bz2

In [ ]:
import sys
!git clone https://github.com/makaveli10/stylegan3
!pip install ninja
sys.path.insert(0, "/content/stylegan3")

## Detect face and crop

In [ ]:
import cv2
import numpy as np
from PIL import Image
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

def find_eyes(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  rects = detector(gray, 0)
  
  if len(rects) == 0:
    raise ValueError("No faces detected")
  elif len(rects) > 1:
    raise ValueError("Multiple faces detected")

  shape = predictor(gray, rects[0])
  features = []

  for i in range(0, 5):
    features.append((i, (shape.part(i).x, shape.part(i).y)))

  return (int(features[3][1][0] + features[2][1][0]) // 2, \
    int(features[3][1][1] + features[2][1][1]) // 2), \
    (int(features[1][1][0] + features[0][1][0]) // 2, \
    int(features[1][1][1] + features[0][1][1]) // 2)

def crop_stylegan(img):
  left_eye, right_eye = find_eyes(img)
  d = abs(right_eye[0] - left_eye[0])
  z = 255/d
  ar = img.shape[0]/img.shape[1]
  w = img.shape[1] * z
  img2 = cv2.resize(img, (int(w), int(w*ar)))
  bordersize = 1024
  img3 = cv2.copyMakeBorder(
      img2,
      top=bordersize,
      bottom=bordersize,
      left=bordersize,
      right=bordersize,
      borderType=cv2.BORDER_REPLICATE)

  left_eye2, right_eye2 = find_eyes(img3)

  crop1 = left_eye2[0] - 385 
  crop0 = left_eye2[1] - 490
  return img3[crop0:crop0+1024,crop1:crop1+1024]

In [ ]:
from matplotlib import pyplot as plt
import cv2
print(SOURCE_NAME)
# image_source = cv2.imread(SOURCE_NAME)
image_source = cv2.imread(SOURCE_NAME)
if image_source is None:
    raise ValueError("Source image not found")

image_target = cv2.imread(TARGET_NAME)
# image_target = cv2.imread(f"./final_results/{TARGET_NAME}")
if image_target is None:
    raise ValueError("Source image not found")

cropped_source = crop_stylegan(image_source)
cropped_target = crop_stylegan(image_target)

img = cv2.cvtColor(cropped_source, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('source')
plt.show()

img = cv2.cvtColor(cropped_target, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('target')
plt.show()

cv2.imwrite("cropped_source.png", cropped_source)
cv2.imwrite("cropped_target.png", cropped_target)

#print(find_eyes(cropped_source))
#print(find_eyes(cropped_target))

## Generate StyleGAN3 Latents

In [ ]:
cmd = f"python /content/stylegan3/projector.py --save-video 0 --num-steps 1000 --outdir={SOURCE} --target=cropped_source.png --network={NETWORK}"
!{cmd}

In [ ]:
cmd = f"python /content/stylegan3/projector.py --save-video 0 --num-steps 1000 --outdir={TARGET} --target=cropped_target.png --network={NETWORK}"
!{cmd}

In [ ]:
img_gan_source = cv2.imread(f'/content/{SOURCE}/proj.png')
img = cv2.cvtColor(img_gan_source, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('source-gan')
plt.show()

In [ ]:
img_gan_target = cv2.imread(f'/content/{TARGET}/proj.png')
img = cv2.cvtColor(img_gan_target, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title('target-gan')
plt.show()

Uncomment if codeformer upsampling needed.

## Merge Faces

In [ ]:
import torch
import dnnlib
import legacy
import PIL.Image
import numpy as np
import imageio
from tqdm.notebook import tqdm

lvec1 = np.load(f'/content/{SOURCE}/projected_w.npz')['w']
lvec2 = np.load(f'/content/{TARGET}/projected_w.npz')['w']

# network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
device = torch.device('cuda')
with dnnlib.util.open_url(NETWORK) as fp:
    G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

diff = lvec2 - lvec1
step = diff / STEPS
current = lvec1.copy()
target_uint8 = np.array([1024,1024,3], dtype=np.uint8)

video = imageio.get_writer('/content/movie.mp4', mode='I', fps=FPS, codec='libx264', bitrate='16M')

for j in tqdm(range(STEPS)):
  z = torch.from_numpy(current).to(device)
  synth_image = G.synthesis(z, noise_mode='const')
  synth_image = (synth_image + 1) * (255/2)
  synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

  repeat = FREEZE_STEPS if j==0 or j==(STEPS-1) else 1
   
  for i in range(repeat):
    video.append_data(synth_image)
  if j == STEPS/2:
    cv2.imwrite(f'./merge_{SOURCE}_{TARGET}.png', cv2.cvtColor(synth_image, cv2.COLOR_RGB2BGR))
  current = current + step


video.close()

In [ ]:
inp_img = f'./merge_{SOURCE}_{TARGET}.png'
cmd = f"python CodeFormer/inference_codeformer.py -w 0.7 --input_path {inp_img} -o ./"
!{cmd}

In [ ]:
download_video = False #@param {type:"boolean"}

from google.colab import files
if download_video: files.download("movie.mp4") 

# SAM - Age transformation

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'SAM'

In [ ]:
!git clone https://github.com/yuval-alaluf/SAM.git $CODE_DIR

In [ ]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 

In [ ]:
os.chdir(f'./{CODE_DIR}')

In [ ]:
from argparse import Namespace
import os
import sys
import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from datasets.augmentations import AgeTransformer
from utils.common import tensor2im
from models.psp import pSp

In [ ]:
EXPERIMENT_TYPE = 'ffhq_aging'

## Step 1: Download Pretrained Model
As part of this repository, we provide our pretrained aging model.
We'll download the model for the selected experiments as save it to the folder `../pretrained_models`.

In [ ]:
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../pretrained_models. """
    current_directory = os.getcwd()
    save_path = os.path.join(os.path.dirname(current_directory), "pretrained_models")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url    

In [ ]:
MODEL_PATHS = {
    "ffhq_aging": {"id": "1XyumF6_fdAxFmxpFcmPf-q84LU_22EMC", "name": "sam_ffhq_aging.pt"}
}

path = MODEL_PATHS[EXPERIMENT_TYPE]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"]) 

In [ ]:
!wget {download_command}

## Step 2: Define Inference Parameters

Below we have a dictionary defining parameters such as the path to the pretrained model to use and the path to the
image to perform inference on.
While we provide default values to run this script, feel free to change as needed.

In [ ]:


EXPERIMENT_DATA_ARGS = {
    "ffhq_aging": {
        "model_path": "../pretrained_models/sam_ffhq_aging.pt",
        "image_path": SOURCE_NAME,
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

## Step 3: Load Pretrained Model
We assume that you have downloaded the pretrained aging model and placed it in the path defined above

In [ ]:
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[EXPERIMENT_TYPE]

In [ ]:
!pwd

In [ ]:
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')

In [ ]:
opts = ckpt['opts']
pprint.pprint(opts)

In [ ]:
# update the training options
opts['checkpoint_path'] = model_path

In [ ]:
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

## Step 4: Visualize Input

In [ ]:
image_path = EXPERIMENT_DATA_ARGS[EXPERIMENT_TYPE]["image_path"]
original_image = Image.open(image_path).convert("RGB")

In [ ]:
original_image.resize((256, 256))

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
def run_alignment(image_path):
    import dlib
    from scripts.align_all_parallel import align_face
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    aligned_image = align_face(filepath=image_path, predictor=predictor) 
    print("Aligned image has shape: {}".format(aligned_image.size))
    return aligned_image 

In [ ]:
aligned_image = run_alignment(image_path)

In [ ]:
aligned_image.resize((256, 256))

In [ ]:
img_transforms = EXPERIMENT_ARGS['transform']
input_image = img_transforms(aligned_image)

In [ ]:
# we'll run the image on multiple target ages 
target_ages = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_transformers = [AgeTransformer(target_age=age) for age in target_ages]

In [ ]:
def run_on_batch(inputs, net):
    result_batch = net(inputs.to("cuda").float(), randomize_noise=False, resize=False)
    return result_batch

In [ ]:
# for each age transformed age, we'll concatenate the results to display them side-by-side
os.makedirs('/content/output_age_images')
for age_transformer in age_transformers:
    print(f"Running on target age: {age_transformer.target_age}")
    with torch.no_grad():
        input_image_age = [age_transformer(input_image.cpu()).to('cuda')]
        input_image_age = torch.stack(input_image_age)
        result_tensor = run_on_batch(input_image_age, net)[0]
        result_image = tensor2im(result_tensor)
        result_image.save(
            f"/content/output_age_images/age_transformed_image-{age_transformer.target_age}.jpg")


In [ ]:
%cd /content

In [ ]:
# @title CodeFormer Upsampling
results = np.array(aligned_image.resize((1024, 1024)))
for age in target_ages:
    print(f"Running on target age: {age}")
    image_path = f"/content/output_age_images/age_transformed_image-{age}.jpg"
    cmd = f"python CodeFormer/inference_codeformer.py -w 0.7 --input_path {image_path} -o ./"
    !{cmd}
    result_image = Image.open(f'./final_results/age_transformed_image-{age}.png').convert("RGB").resize((1024, 1024))
    results = np.concatenate([results, result_image], axis=1)

In [ ]:
# save image at full resolution
final_results = Image.fromarray(results)
final_results.save("age_transformed_image.jpg")